原创教程，版权所有。

同济子豪兄B站视频专栏：https://space.bilibili.com/1900783

玩转UCI心脏病二分类数据集，课件、代码、答疑互动：https://t.zsxq.com/Z7yNZBu

子豪兄Python交流QQ群：1077638784

子豪兄Kaggle数据科学竞赛交流：481041896

微信公众号：人工智能小技巧

2020-05-15


# 本节概述

在uci心脏病数据集上训练模型之后，对测试集进行预测，计算测试集每个样本的每个特征对两类预测结果的shap值。

为后续shap值可视化，绘制决策图奠定基础。

# 构建数据集、训练模型、对测试集样本预测

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

# 忽略烦人的红色提示
import warnings
warnings.filterwarnings("ignore")

# 忽略烦人的红色提示
import warnings
warnings.filterwarnings("ignore")

# 导入数据集，划分特征和标签
df = pd.read_csv('process_heart.csv')
X = df.drop('target',axis=1)
y = df['target']

# 划分训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# 构建随机森林模型
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=5, n_estimators=100)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_pred_proba = model.predict_proba(X_test)

# 导入shap机器学习可解释性分析工具包

In [2]:
import shap
shap.initjs()

# 计算测试集每个样本的每个特征对两类预测结果的shap值

In [3]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

In [4]:
len(shap_values)

2

In [5]:
# 61个样本，26个特征，对“不患病”预测结果的shap值
shap_values[0].shape

(61, 26)

In [6]:
# 61个样本，26个特征，对“患病”预测结果的shap值。
shap_values[1].shape

(61, 26)

In [7]:
shap_values[1]

array([[ 2.26305672e-03, -6.35933340e-03, -2.58501306e-02, ...,
         8.31911930e-04, -4.74062520e-02, -4.80973552e-04],
       [-1.45180434e-02,  2.37988985e-03,  1.50114880e-02, ...,
         2.99710491e-03, -3.47719651e-02,  1.36389951e-05],
       [-1.02876868e-02,  9.58861568e-03, -2.73919829e-02, ...,
         1.40991912e-03, -4.95074693e-02,  7.31445901e-05],
       ...,
       [ 8.48085641e-03, -3.31496921e-03, -2.32143866e-03, ...,
         1.01051160e-03, -3.54479873e-02, -2.14148412e-05],
       [-7.93840434e-03, -6.16357834e-03,  1.84116224e-03, ...,
         7.37899174e-04, -3.83237400e-02, -1.73224184e-04],
       [ 5.63130152e-03, -2.18797898e-02,  2.80598171e-02, ...,
         3.46359033e-04, -6.65968124e-02,  1.84694601e-04]])

# 测试集所有样本，预测为“不患病”和“患病”各自的平均概率

In [8]:
explainer.expected_value

array([0.42392562, 0.57607438])

In [9]:
explainer.expected_value[0] + explainer.expected_value[1]

0.9999999999999998

# 对某个样本，模型预测为“患病”的概率即为
# `测试集“患病”的平均概率` 与 `该样本各特征对“患病”预测结果的shap值` 之和。

对某个样本，模型预测“患病”的概率即为explainer.expected_value[1] 与 该样本各特征shap值之和。

In [10]:
shap_values[1].shape

(61, 26)

In [11]:
shap_values[1].sum(axis=1).shape

(61,)

In [12]:
y_pred_proba_shap = shap_values[1].sum(axis=1) + explainer.expected_value[1]

In [13]:
y_pred_proba_shap.shape

(61,)

In [14]:
y_pred_proba_shap

array([0.0516996 , 0.40823446, 0.37500563, 0.78406802, 0.30587708,
       0.73789519, 0.46106359, 0.61887753, 0.87970051, 0.10672854,
       0.88216688, 0.20567386, 0.59998603, 0.8325713 , 0.18001561,
       0.80826619, 0.13555421, 0.01950697, 0.5717426 , 0.28885471,
       0.083452  , 0.8014563 , 0.68104344, 0.9001789 , 0.27357706,
       0.25434244, 0.02945419, 0.06957194, 0.88900142, 0.01141173,
       0.85778153, 0.31056465, 0.1903174 , 0.20491476, 0.0119005 ,
       0.12089845, 0.75488439, 0.56437276, 0.82922519, 0.23176921,
       0.06049483, 0.23659182, 0.81544692, 0.55689614, 0.89469141,
       0.47065697, 0.53074936, 0.86114929, 0.72390046, 0.39292512,
       0.57038257, 0.8231123 , 0.04002881, 0.16629465, 0.94054856,
       0.8027777 , 0.81714106, 0.79867226, 0.0218079 , 0.01463372,
       0.65801722])

In [15]:
model.predict_proba(X_test)

array([[0.9483004 , 0.0516996 ],
       [0.59176554, 0.40823446],
       [0.62499437, 0.37500563],
       [0.21593198, 0.78406802],
       [0.69412292, 0.30587708],
       [0.26210481, 0.73789519],
       [0.53893641, 0.46106359],
       [0.38112247, 0.61887753],
       [0.12029949, 0.87970051],
       [0.89327146, 0.10672854],
       [0.11783312, 0.88216688],
       [0.79432614, 0.20567386],
       [0.40001397, 0.59998603],
       [0.1674287 , 0.8325713 ],
       [0.81998439, 0.18001561],
       [0.19173381, 0.80826619],
       [0.86444579, 0.13555421],
       [0.98049303, 0.01950697],
       [0.4282574 , 0.5717426 ],
       [0.71114529, 0.28885471],
       [0.916548  , 0.083452  ],
       [0.1985437 , 0.8014563 ],
       [0.31895656, 0.68104344],
       [0.0998211 , 0.9001789 ],
       [0.72642294, 0.27357706],
       [0.74565756, 0.25434244],
       [0.97054581, 0.02945419],
       [0.93042806, 0.06957194],
       [0.11099858, 0.88900142],
       [0.98858827, 0.01141173],
       [0.

In [16]:
model.predict_proba(X_test)[:,1]

array([0.0516996 , 0.40823446, 0.37500563, 0.78406802, 0.30587708,
       0.73789519, 0.46106359, 0.61887753, 0.87970051, 0.10672854,
       0.88216688, 0.20567386, 0.59998603, 0.8325713 , 0.18001561,
       0.80826619, 0.13555421, 0.01950697, 0.5717426 , 0.28885471,
       0.083452  , 0.8014563 , 0.68104344, 0.9001789 , 0.27357706,
       0.25434244, 0.02945419, 0.06957194, 0.88900142, 0.01141173,
       0.85778153, 0.31056465, 0.1903174 , 0.20491476, 0.0119005 ,
       0.12089845, 0.75488439, 0.56437276, 0.82922519, 0.23176921,
       0.06049483, 0.23659182, 0.81544692, 0.55689614, 0.89469141,
       0.47065697, 0.53074936, 0.86114929, 0.72390046, 0.39292512,
       0.57038257, 0.8231123 , 0.04002881, 0.16629465, 0.94054856,
       0.8027777 , 0.81714106, 0.79867226, 0.0218079 , 0.01463372,
       0.65801722])